In [48]:
import tensorflow as tf

import numpy as np
import os
import time

# Praparing data

In [49]:
from glob import glob

files = glob(f'../input/poemsdataset/**/**/*.txt')

In [50]:
files[:5], len(files)

(['../input/poemsdataset/forms/lay/LayPoemsLayAGarlandOnMyHearsePoembyFrancisBeaumont.txt',
  '../input/poemsdataset/forms/lay/LayPoemsLayHisSwordByHisSidePoembyThomasMoore.txt',
  '../input/poemsdataset/forms/lay/LayPoemsLayAGarlandOnMyHearsePoembyBeaumontandFletcher.txt',
  '../input/poemsdataset/forms/lay/LayPoemsAsILayWithHeadInYourLapCameradoPoembyWaltWhitman.txt',
  '../input/poemsdataset/forms/lay/LayPoemsTheDeerLayDownTheirBonesPoembyRobinsonJeffers.txt'],
 20657)

In [51]:
text = [open(f).read() for f in files]
print('length of text', len(text))
text[:3]

length of text 20657


['Lay a garland on my hearse,\nOf the dismal yew,\nMaidens, willow branches bear,\nSay I died true.\nMy love was false, but I was firm\nFrom my hour of birth;\nUpon my buried body lie\nLightly, gentle earth.',
 'Lay his sword by his side -- it hath served him too well\nNot to rest near his pillow below;\nTo the last moment true, from his hand ere it fell,\nIts point was still turn\'d to a flying foe.\nFellow-labourers in life, let them slumber in death,\nSide by side, as becomes the reposing brave --\nThat sword which he loved still unbroke in its sheath,\nAnd himself unsubdued in his grave.\nYet pause -- for, in fancy, a still voice I hear,\nAs if breathed from his brave heart\'s remains; --\nFaint echo of that which, in Slavery\'s ear,\nOnce sounded the war-word, "Burst your chains."\nAnd it cries, from the grave where the hero lies deep,\n"Though the day of your Chieftain for ever hath set,\nOh leave not his sword thus inglorious to sleep --\nIt hath victory\'s life in it yet!\n"Sho

In [52]:
text = ''.join(text)

In [53]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

1755 unique characters


# Encoding data

In [54]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [55]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [56]:
# testing 

example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print('chars:', chars)

ids = ids_from_chars(chars)
print('id:', ids)

chars = chars_from_ids(ids)
print('chars:', chars)

chars: <tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>
id: <tf.RaggedTensor [[69, 70, 71, 72, 73, 74, 75], [92, 93, 94]]>
chars: <tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [57]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# Create training examples and targets

In [58]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(20928289,), dtype=int64, numpy=array([48, 69, 93, ..., 35,  4, 98])>

In [59]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [60]:
# test

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

L
a
y
 
a
 
g
a
r
l


In [61]:
seq_length = 25
examples_per_epoch = len(text)//(seq_length+1)

In [62]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'L' b'a' b'y' b' ' b'a' b' ' b'g' b'a' b'r' b'l' b'a' b'n' b'd' b' '
 b'o' b'n' b' ' b'm' b'y' b' ' b'h' b'e' b'a' b'r' b's' b'e'], shape=(26,), dtype=string)


2022-03-18 06:39:40.504472: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


In [63]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'Lay a garland on my hearse'
b',\nOf the dismal yew,\nMaide'
b'ns, willow branches bear,\n'
b'Say I died true.\nMy love w'
b'as false, but I was firm\nF'


In [64]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [65]:
split_input_target(list("I love python!"))

(['I', ' ', 'l', 'o', 'v', 'e', ' ', 'p', 'y', 't', 'h', 'o', 'n'],
 [' ', 'l', 'o', 'v', 'e', ' ', 'p', 'y', 't', 'h', 'o', 'n', '!'])

In [66]:
dataset = sequences.map(split_input_target)

In [67]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Lay a garland on my hears'
Target: b'ay a garland on my hearse'


# Create training batches

In [68]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 25), (64, 25)), types: (tf.int64, tf.int64)>

# Build The Model

In [69]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [70]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [71]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [72]:
# testing

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 25, 1756) # (batch_size, sequence_length, vocab_size)


In [73]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  449536    
_________________________________________________________________
gru_1 (GRU)                  multiple                  3938304   
_________________________________________________________________
dense_1 (Dense)              multiple                  1799900   
Total params: 6,187,740
Trainable params: 6,187,740
Non-trainable params: 0
_________________________________________________________________


In [74]:
# testing

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print(sampled_indices)
print()
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

[ 357 1147  208  285 1131 1562 1719  868  958  455 1108 1203 1123 1646
  102 1196  208  610  664  692  974 1521  344  753 1243]

Input:
 b' full long.\nThe porter hu'

Next Char Predictions:
 b'\xe0\xa4\x85\xe6\x9d\x9f\xce\x93\xd0\xbb\xe6\x9c\x83\xe8\xaf\x97\xe9\xa5\xb6\xe5\xa2\xa8\xe5\xb7\xb1\xe0\xa6\xab\xe6\x97\xa0\xe6\xb0\xb8\xe6\x9a\x96\xe9\x87\x8e\xc2\xa4\xe6\xaf\x9b\xce\x93\xe2\x98\x86\xe4\xba\xa8\xe4\xbd\x93\xe5\xb9\xbf\xe8\x99\x9a\xd9\xbe\xe5\x8a\x87\xe6\xb5\x8b'


# Training model

In [75]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 25, 1756)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(7.471012, shape=(), dtype=float32)


1756.3834

In [76]:
model.compile(optimizer='adam', loss=loss)

In [77]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [78]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
12577/12577 [==============================] - 188s 15ms/step - loss: 1.7389
Epoch 2/20
12577/12577 [==============================] - 192s 15ms/step - loss: 1.5851
Epoch 3/20
12577/12577 [==============================] - 191s 15ms/step - loss: 1.5580
Epoch 4/20
12577/12577 [==============================] - 187s 15ms/step - loss: 1.5450
Epoch 5/20
12577/12577 [==============================] - 187s 15ms/step - loss: 1.5397
Epoch 6/20
12577/12577 [==============================] - 190s 15ms/step - loss: 1.5372
Epoch 7/20
12577/12577 [==============================] - 186s 15ms/step - loss: 1.5366
Epoch 8/20
12577/12577 [==============================] - 187s 15ms/step - loss: 1.5404
Epoch 9/20
12577/12577 [==============================] - 188s 15ms/step - loss: 1.5475
Epoch 10/20
12577/12577 [==============================] - 187s 15ms/step - loss: 1.5733
Epoch 11/20
12577/12577 [==============================] - 186s 15ms/step - loss: 1.6032
Epoch 12/20
12577/12577 [=====

In [84]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [85]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [86]:
start = time.time()
states = None
next_char = tf.constant(['In the heaven'])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

In the heaven --
Whappye sho hy somatel pows
Dow vef theiv inghro ofhe piped tam fuchat thies han ' oure Bous theh bif
I mame, youro gural Phop.
Thotounide walt t hit exery,
F math mores thuribl cripdwl tore
Nlickid thived crene?memberess the povedesseove
Y shes coce ofhre hark now ther yoo
And the slar;
St Ther beling meansilo willslideng to athing flliw
Curd!
Ho aresce fiveli!
Th add th tmer crethon” tofeophs aggnos went ino the prowm,
ant chtrock ot loerding thre k're.
P thers oove tou he lestr.
I. loceath freer-shechand shoul you't hal thitt;h amis.....
Toret yoly ouren youldiourameithished erynelats winthis thid the Lue its ilatid,
Who ghancat qumes I’2lpe po listig and hou ovinge as wor,
Thtuck age ut the begltive shipidhem o blat d
Hover.
'illd beld a fter bighor, aon mor'st hik "
Fant  on thinept.
Ce carse sapis I che meall shetilie
SJulted.
I fases orem bh
Te fe herdaist witi for.
onst allat they,
Asth g;ura beembblf a cone uatPe berke to
Bime o  ttatଥed ofάυιι（ tπDy tईeekher 

In [87]:
start = time.time()
states = None
next_char = tf.constant(['I see the light'])
result = [next_char]

for n in range(100):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

I see the light,
The parmomout of btesh to sande
Donem
Thy thiapest the fre chor bric, as ande ime mecieroug it til 

________________________________________________________________________________

Run time: 0.20508980751037598


In [88]:
start = time.time()
states = None
next_char = tf.constant(["I will meet you again"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

I will meet you againt 20202008e of Pusid yourea by art stelpe
Or cht onecturng nordngt-chotl there with.
N whe bllies
Weiygeme inth that you h ssur?
Howan lol wokresese ble co awer!
Whee the st wavberss halld sanc hpre dswither seelow finat ook un sucet,
Sh thoof dre aw there Bersple the cathe onepe shet doope
Whe.
Out cherel tog bo leit pathers borref wighistas dast all norgthers bres hes bhloochcherfol,e Grom agtoon,sple all thavy siile, way krindghe the upis of one, hit fisee hee mus y itames Ingrowl
ang had hovere!
Whover go the hiou
Ther --sret
shat trlat
Os bringho deptheatal
 roult? heme'nd string to ge she
Oun we them
J the Iy un chong,
Foeg,
Id mucho furets int not doon't thers fare,
What the Catibld huntie
'l'll ayou ever your too cellld ther itheer bacoly,
I
Blintof any un tore wone hipuit heselce ourt ithesed mely inarth
ion prelth-tere isit gor bre liridg tthe hearthe althe 's sker onentas'
Teaus, dre cueamp m
 mere th hot ales to rod neope ct mese, In'te shatter thoog be